# **YOLOv4 on Build Sense** 

## Instructions to run Google Colab

1. Connect Runtime to GPU for better/faster results [RunTime -> Change RunTime Type -> GPU]
2. Download the custom dataset and the annotations.
3. Upload the custom dataset and the annotations to the Google Drive account connected to this Google Colab Notebook.

## 0. Prerequisites

In [ ]:
# Check whether GPU is provided
!nvidia-smi
!nvcc --version

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages and restart runtime (if required)
!pip install opencv-python --upgrade

In [ ]:
# Check Python Version and import os to evaluate current directories
!python --version
import os

## 1. Setup YOLOv4 and Custom Dataset

### 1.1. Clone the required repositories

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Clone the required repositories
!git clone https://github.com/AlexeyAB/darknet.git    # Official YOLOv4 Implementation
!git clone https://github.com/adityapujar1/build-sense.git      # Build Sense repository

### 1.2. Update Makefile based on requirements

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Update the makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

### 1.3. Build Darknet

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Build Darknet
!make

### 1.4. Download Pre-trained weights file

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Download the pretrained weights file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137     # For training cfg/yolov4-custom.cfg
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29     # For training cfg/yolov4-tiny-custom.cfg

# Change Directory
%cd ..

### 1.5. Setup the Custom Dataset

**Important:** Please fulfill the following conditions to use the custom dataset:
1. Download the Images and the Labels and upload the zip folders into the Google Drive connected to this Colab Notebook.

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Sanity Check: Check if the Custom Dataset and the annotations exists in the Google Drive connected to this Google Colab Network
assert os.path.exists('./drive/MyDrive/yolo_custom_model_training/build-sense.zip'), 'Custom Dataset does not exist! Please follow the instructions from the cell above!'

# Copy the Dataset and the Annotations
!cp -rvi ./drive/MyDrive/yolo_custom_model_training/build-sense.zip ./darknet/data/

# Change Current Directory
%cd darknet

# Unzip the Dataset and the Annotations in the data folder
!unzip ./data/build-sense.zip -d ./data/

# Delete unwanted files
!rm -rf ./data/build-sense.zip

# Check Contents of the Current Directory
print("\nThe contents of the {} directory are:".format(os.getcwd()))
!ls

# Count the number of training, validation and test images
print("Number of training images: {}".format(len(os.listdir('./data/build-sense/train/'))/2))
print("Number of validation images: {}".format(len(os.listdir('./data/build-sense/test/'))/2))

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Copy build-sense.names files
!cp -vi ../build-sense/data/* ./data/build-sense/

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Generate Paths to Training and Test Images
!python ../build-sense/utils/generate_paths.py -it data/build-sense/train/ -iv data/build-sense/test/ -o ./data/build-sense/

# Generate data file containing relative paths to the training, validation and backup folders for YOLOv4
!python ../build-sense/utils/generate_data_file.py -c 3 -t data/build-sense/build_sense_train.txt -v data/build-sense/build_sense_test.txt -n data/build-sense/build_sense.names -b backup/ -o ./data/build-sense/

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Copy pre-defined YOLOv4 network config file to cfg folder
!cp -vi ../build-sense/config/* ./cfg/

### 1.6. Visualize Samples from Custom Dataset

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Import required libraries
import glob
import matplotlib.pyplot as plt

def read_annotations(imagePath):
  annotationsPath = imagePath.replace('.png', '.txt')
  with open(annotationsPath, 'rt') as annotationsFile:
    print(os.path.basename(annotationsPath) + ': \n\n' + annotationsFile.read())

imagePath = glob.glob("./data/build-sense/train/*.png")
numberOfSamples = 2       # Can be changed based on requirements

for sampleIterator in range(numberOfSamples):
  fig = plt.figure(figsize=(20,20))
  sampleImage = plt.imread(str(imagePath[sampleIterator]))
  plt.axis(False)
  plt.imshow(sampleImage)
  read_annotations(str(imagePath[sampleIterator]))
  plt.savefig("dataset_sample_{}".format(sampleIterator), dpi=300)

### 1.7. Create Symbolic Link to Google Drive

**Important:** This step is important to ensure that the weights are stored in our drive.

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Delete Backup folder
!rm -rf ./backup

# Create a New Backup Folder
!mkdir backup

# Create Symbolic links so we can save trained weights file in our Google Drive
# Create the folder YOLOv4_weight/backup in your Drive to store trained weights
!ln -svt /content/drive/MyDrive/yolo_custom_model_training/YOLOv4_weight/ /content/darknet/backup/

## 2. Train YOLOv4 on Custom Dataset

### 2.1. Train YOLOv4 on Berkley DeepDrive Dataset

**Important:**
1. If you get CUDA out of memory error, adjust the number of sub-divisions in the config file
2. Adjust the number of max batches for shorter training time

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Train YOLOv4 on Custom Dataset (Berkley DeepDrive Dataset)
!./darknet detector train ./data/build-sense/build_sense.data ./cfg/yolov4-tiny-build-sense.cfg ./yolov4-tiny.conv.29 -dont_show -map

### 2.2. Visualize Training Results

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Plot Training Results
fig = plt.figure(figsize=(20,20))
trainingResult = plt.imread("chart.png")
plt.axis(False)
plt.imshow(trainingResult)

## 3. Test YOLOv4 on Custom Dataset

### 3.1. Setup test images and test video

**Important:** Please fulfill the following conditions to use the Custom Dataset:
1. Download the Images and the Video Parts 
2. Compress the test images and test videos into a folder and upload the zip folders into the Google Drive connected to this Colab Notebook.

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Sanity Check: Check if the test images and test videos exists in the Google Drive connected to this Google Colab Network
%cd ..
assert os.path.exists('./drive/MyDrive/yolo_custom_model_training/test.zip'), 'Test images and test videos does not exist! Please follow the instructions from the cell above!'

# Copy the test images and test videos
!cp -rvi ./drive/MyDrive/yolo_custom_model_training/test.zip .

# Unzip the Dataset and the Annotations in the data folder
!unzip ./test.zip -d ./test

# Delete unwanted files
!rm -rf ./test.zip

# Check Contents of the test Directory
%cd test
print("\nThe contents of the {} directory are:".format(os.getcwd()))
!ls

# Change current directory
%cd ..

### 3.2. Setup Weights, Configuration, Labels, Confidence Thresholds and Non-Maxmimum Suppression Thresholds

**Important:** 
1. In order to test YOLOv4 on an example image or example video, the path to the weights file, path to the configuration file, path to the labels file, confidence thresholds and non-maximum thresholds must be specified. Confidence thresholds are required to discard low confidence detections, and non-maximum suppression thresholds are needed to discard false positives. 
2. Currently, the paths specified in the below cell point to the files that are already available in the ***build-sense*** repository. However, the paths and the threshold values can be changed to point to different files and threshold values.

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Change current directory
%cd ./build-sense

# Clear Output directory
!rm -rf ./output

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/build-sense', 'Directory should be "/content/build-sense" instead of "{}"'.format(os.getcwd())

# Specify path to the weights file, path to the configuration file, path to the labels file, confidence thresholds and non-maximum thresholds (Can be changed if required)
nnWeights = "./weights/yolov4-tiny-build-sense_best.weights"
nnConfiguration = "./config/yolov4-tiny-build-sense.cfg"
labelsPath = "./data/build_sense.names"
confidenceThreshold = 0.5
nmsThreshold = 0.3

### 3.3. Test YOLOv4 on an example image

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/build-sense', 'Directory should be "/content/build-sense" instead of "{}"'.format(os.getcwd())

# Specify image path
imagePath = "../test/brick_wall_vertical_cracks17.png"

# Test YOLOv4 on an example image
!python detection.py --nnWeights $nnWeights --nnConfiguration $nnConfiguration --labelsPath $labelsPath --confidenceThreshold $confidenceThreshold --nmsThreshold $nmsThreshold --imagePath $imagePath

# View YOLOv4 Output on the example image
fig = plt.figure(figsize=(20,20))
yolov4ImageOuput = plt.imread("./output/brick_wall_vertical_cracks17.png")
plt.axis(False)
plt.imshow(yolov4ImageOuput)


## Congratulations! You have successfully trained the YOLOv4 network on a Custom dataset and tested the YOLOv4 network on test images